# Autoencoder for network data

Autoencoder is a non-linear dimensionality reduction tool (deep representation learning). It is an artificial neural network, or actually two neural networks being trained simultaneously:  
- first half of the layers or first network encodes the input high-dimensional data into lower-dimensional representation space, gradually reducing dimensionality from layer to layer;  
- second half transforms it back from the low-dimensional encoded representation feature space to the original dimensionality.

The architectures of the two parts are usually symmetric as could be seen on the figure below

<img src="autoencoder.png">

Two useful blogs for getting started:  

https://ramhiser.com/post/2018-05-14-autoencoders-with-keras/

https://www.datacamp.com/community/tutorials/autoencoder-keras-tutorial  

https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798

(credits for the figure to the third one)

In [1]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras import regularizers
import pickle

Using TensorFlow backend.


**The purpose of the below is to read a temporal network (each row is a time-slice, columns are edge weights) and run autoencoder to convert it to a smaller space of representative feature timelines**

In [2]:
def save(filename,X):
    outfile = open(filename,'wb')
    pickle.dump(X,outfile)
    outfile.close()

In [3]:
def load(filename):
    infile = open(filename,'rb')
    X=pickle.load(infile)
    infile.close()
    return X

In [4]:
X=load('Taipeiexchange1.pkl')

In [5]:
x_test=X[400:,0:100]; x_train=X[0:400,0:100]; #split the data into test and train

In [32]:
#design autoencoder
input_dim = x_train.shape[1]
#encoding_dim = [50,15,5,2] #sizes of the layers
#encoding_dim = [30,10,3] #sizes of the layers
encoding_dim = [80,60,40,25,15] #sizes of the layers
#encoding_dim = [50,25,12] #sizes of the layers

compression_factor = float(input_dim) / encoding_dim[-1]
print("Compression factor: %s" % compression_factor)

#define autoencoder architecture
autoencoder = Sequential()
autoencoder.add(
    Dense(encoding_dim[0], input_shape=(input_dim,), activation='relu')
)

for l in range(1,len(encoding_dim)):
    autoencoder.add(
         Dense(encoding_dim[l], input_shape=(encoding_dim[l-1],), activation='relu')
    )
    
for l in range(len(encoding_dim)-1,0,-1):
    autoencoder.add(
         Dense(encoding_dim[l-1], input_shape=(encoding_dim[l],), activation='relu')
     )

autoencoder.add(
    Dense(input_dim, input_shape=(encoding_dim[0],), activation='sigmoid')
)

autoencoder.summary()

Compression factor: 6.66666666667
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_32 (Dense)             (None, 60)                4860      
_________________________________________________________________
dense_33 (Dense)             (None, 40)                2440      
_________________________________________________________________
dense_34 (Dense)             (None, 25)                1025      
_________________________________________________________________
dense_35 (Dense)             (None, 15)                390       
_________________________________________________________________
dense_36 (Dense)             (None, 25)                400       
_________________________________________________________________
dense_37 (Dense)             (None, 40)   

In [33]:
np.random.seed(0)

In [34]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [35]:
autoencoder.fit(x_train, x_train,
                epochs=400,
                batch_size=50,
                shuffle=True,
                validation_data=(x_test, x_test),verbose=1)

Train on 400 samples, validate on 269 samples
Epoch 1/400
400/400 [==============================] - 2s 5ms/step - loss: 0.6921 - val_loss: 0.6893
Epoch 2/400
400/400 [==============================] - 0s 131us/step - loss: 0.6834 - val_loss: 0.6708
Epoch 3/400
400/400 [==============================] - 0s 127us/step - loss: 0.6538 - val_loss: 0.6338
Epoch 4/400
400/400 [==============================] - 0s 126us/step - loss: 0.6249 - val_loss: 0.6083
Epoch 5/400
400/400 [==============================] - 0s 126us/step - loss: 0.6064 - val_loss: 0.5964
Epoch 6/400
400/400 [==============================] - 0s 129us/step - loss: 0.5966 - val_loss: 0.5870
Epoch 7/400
400/400 [==============================] - 0s 127us/step - loss: 0.5872 - val_loss: 0.5806
Epoch 8/400
400/400 [==============================] - 0s 134us/step - loss: 0.5820 - val_loss: 0.5767
Epoch 9/400
400/400 [==============================] - 0s 130us/step - loss: 0.5789 - val_loss: 0.5749
Epoch 10/400
400/400 [=======

In [36]:
#extract encoder layers from the networks
def encoderLayers(input_):
    output_=input_
    for l in range(len(encoding_dim)):
        output_=autoencoder.layers[l](output_)
    return output_

input_layer = Input(shape=(input_dim,))
#encoder_layer = autoencoder.layers[0]
encoder = Model(input_layer, (encoderLayers(input_layer)))

encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 80)                8080      
_________________________________________________________________
dense_32 (Dense)             (None, 60)                4860      
_________________________________________________________________
dense_33 (Dense)             (None, 40)                2440      
_________________________________________________________________
dense_34 (Dense)             (None, 25)                1025      
_________________________________________________________________
dense_35 (Dense)             (None, 15)                390       
Total params: 16,795
Trainable params: 16,795
Non-trainable params: 0
_________________________________________________________________


In [37]:
#encode the data
encoded_x = encoder.predict(x_test)
decoded_x = autoencoder.predict(x_test)

In [38]:
y=encoder.predict(np.concatenate((x_train,x_test)))

In [39]:
y[:10]

array([[ 1.3363421 , -0.        ,  0.78147084, -0.        ,  5.585493  ,
         1.2504551 , -0.        , -0.        ,  0.01923567, -0.        ,
         1.7564486 ,  4.062417  ,  3.8543825 ,  2.4002512 ,  1.2071981 ],
       [ 0.7709331 , -0.        ,  2.8277085 , -0.        ,  3.6389208 ,
         1.0418524 , -0.        , -0.        , -0.        , -0.        ,
         0.8897324 ,  4.085449  ,  4.341785  ,  2.5373921 ,  2.4504175 ],
       [-0.        , -0.        ,  4.1625543 , -0.        ,  0.4345499 ,
         0.46726698, -0.        , -0.        , -0.        , -0.        ,
         1.0089458 ,  3.8056495 ,  1.9283712 ,  5.624664  ,  2.9422386 ],
       [-0.        , -0.        ,  4.468466  , -0.        ,  0.5748289 ,
         0.2916113 , -0.        , -0.        , -0.        , -0.        ,
         1.2474947 ,  3.8932695 ,  1.8872777 ,  6.0076885 ,  3.491469  ],
       [-0.        , -0.        ,  4.6276383 , -0.        ,  0.8261027 ,
         0.589389  , -0.        , -0.        , 

In [40]:
#save the outcome
save('Tapeiexchange2.pkl',y)